In [11]:
#Importing required packages
import pandas as pd
import numpy as np
import panel as pn
import panel.widgets as pnw
import hvplot as hv
import hvplot.pandas
import hvplot.xarray
import xarray as xr

#Enabling panel displays
pn.extension(sizing_mode = 'stretch_width')

In [12]:
#Importing extracted MTO data 
MTOparam_ds = xr.open_dataset("MTOparam.nc")

In [13]:
#Importing extracted temperature data
IDFdata = xr.open_dataset("Relevant_Temp_Data.nc")

In [14]:
#About tab text, exported from document using extension that converts google docs to markdown
about = pn.pane.Markdown("""
Climate change is expected to cause an increase in the frequency and intensity of extreme precipitation events. As a result, historic Intensity-Duration-Frequency (IDF) data must be adjusted to match future climate conditions in order to reduce risk to infrastructure, public safety, properties and natural systems. 

This tool was created to facilitate the direct comparison of MTO’s current climate change adjustment method against a temperature scaling method suggested by Climatedata Canada under different emissions scenarios. The different methods result in a low, medium and high climate change adjustment corresponding to MTO’s current method, moderate emissions temperature scaling, and high emissions temperature scaling respectively. Intended to allow designers to select which factor to use through a risk assessment matrix.\n

**Climate Change Adjustment Methods**

<span style="text-decoration:underline;">MTO: Extrapolation of Trends</span>

MTO currently projects future IDF values using an extrapolation of trends. A time trend analysis was performed on yearly rainfall data from 1960 to 2014 using combined data from all station datasets. Through this analysis, a collective linear trend was observed in rainfall intensity by year and this linear adjustment factor, _m_, was extrapolated to future IDF datasets. The MTO tool uses 2010 as the base year and applies this linear adjustment factor, _m_, to predict future rainfall intensity data, RR<sub>y</sub> (mm/h), in year y > 2010 as shown in Equation 1 below.

_RR<sub>y</sub> = RR<sub>2010</sub> + m(y - 2010) 									        (1)_

The factor _m_ varies only with duration and is modeled by Equation 2 below:

_m = 0.0279(duration)<sup>-0.494</sup> 									        (2)_

Since this analysis combines all station datasets, it does not address the spatial variability of extreme precipitation time trends within Ontario. Furthermore, this method assumes that the rate of change observed in historical data, _m_, will remain constant.

In order to directly compare this climate change adjustment method against the temperature scaling method, the adjustment is converted to a multiplier as shown in Equation 3:

_Multiplier = [RR<sub>2010</sub> + m(y - 2010)]/RR<sub>2010							              </sub>(3)_

<span style="text-decoration:underline;">Temperature Scaling </span>

Climate Data Canada currently suggests using a temperature scaling method to adjust historic IDF data. This method is based on the Clausius Clapeyron relationship which governs the capacity of air to hold moisture. Future rainfall intensity (RR<sub>y</sub>) can be found by adjusting historic rainfall intensity (RR) with the projected change in temperature (ΔT) between years as shown in Equation 4. 

_RR<sub>y</sub> = RR*1.07<sup>ΔT</sup>										        (4)_

The change in temperature is calculated by finding the difference between the 31y average of projected mean annual temperatures centered around the end year and the start year. To directly compare this adjustment with MTO’s method, the start year is set to 2010. This method is used in the tool to adjust rainfall under a moderate emissions scenario (RCP 4.5) and a high emissions scenario (RCP 8.5).

""")

In [15]:
#Tool overview text
overview = pn.Card(pn.pane.Markdown("""
Extreme precipitation has been found to be increasing in frequency and intensity due to climate change. As a result, 
Intensity-Duration-Frequency (IDF) curves based solely on historical data need to be adjusted in order to accurately
predict future rainfall extremes. \n

This tool facilitates the comparison of the current climate change adjustment method used by the Ministry of Transportation
of Ontario (MTO) which is incorporated in the [MTO IDF Curve Look-up Tool](http://www.mto.gov.on.ca/IDF_Curves/map_acquisition.shtml), 
against a [temperature scaling method recommended by Climate Data Canada](https://climatedata.ca/resource/idf-curves-and-climate-change/)
which adjusts rainfall intensity with the projected change in temperature. As the existing MTO tool uses 2010 as the base 
data year, all adjustments are presented as multipliers to MTO 2010 rainfall intensity data to allow for direct comparison
between methods.\n

<span style="text-decoration:underline">Climate Change Adjustment Labels Used in Tool</span> \n
* **Multiplier: **A climate change adjustment factor which can be multiplied with the base 2010 rainfall intensity to predict
a future rainfall intensity. For example: <br>
_2072 Low Rainfall Intensity = (MTO 2010 Rainfall Intensity)x(2072 Low Intensity Multiplier)_
* **MTO: **Represents the linear climate change adjustment MTO currently uses to predict future rainfall intensities.
* **Low:** Represents a climate change adjustment related to the projected change in temperature under a moderate emissions scenario (RCP 4.5).
* **High:** Represents a climate change adjustment method related to projected change in temperature under a high emissions scenario (RCP 8.5).

View how a variable changes across the province in the Choropleth Map. See a summary of the different projected rainfall 
extremes at a selected location in the Summary Table. Control both the map and the summary table through the 
control panel to the left.

For more information, select the 'About' tab above.
"""),title = "Overview")

#overview.show()  #uncomment to see formatting inidividually

In [16]:
#Generating interactive widgets
returnperiod = pnw.Select(name = 'Select Return Period (years)',options = ['2','5','10','25','50','100'],width = 225)
duration = pnw.FloatInput(name='Input Duration (hours)',value=1, start = 0, end = 25,width = 225)
endyear = pnw.IntInput(value = 2022, start = 2022, end = 2085, name = 'Select End Year', width = 225)
lat = pnw.FloatInput(name='Select Latitude',value=54.3829, start = 41.6, end = 57.984,width = 225)
lon = pnw.FloatInput(name='Select Longitude', value = -85.4670, start = -95.96,end = -74.0, width = 225)
var_names = ['MTO2010','dT_rcp45', 'dT_rcp85','af_MTO','af_rcp45', 'af_rcp85']
var_userfriendly = ['MTO 2010 Rainfall Intensity','Moderate Projected Temperature Change','High Projected Temperature Change','MTO Intensity Multiplier','Low Intensity Multiplier','High Intensity Multiplier']
#varoptions = [v for v in list(IDFdata.data_vars.keys()) if "laf" not in v] + ['af_MTO']
var = pn.widgets.Select(options = var_userfriendly, name = 'Select Variable to Display on Map', width = 225)

In [17]:
# Formatting and previewing Control Panel 
control_panel_header = pn.pane.Markdown("""
### **Control Panel** 
Use the widgets below to control the Choropleth Map and Summary Table to the right. """)
control_panel = pn.Column(control_panel_header,var,endyear,duration,returnperiod,lat,lon,max_width = 400)
control_panel

Column(max_width=400, sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Select(options=['MTO 2010 Rainfall Intens...], sizing_mode='stretch_width', value='MTO 2010 Rainfall I..., width=225)
    [2] IntInput(end=2085, name='Select End Year', sizing_mode='stretch_width', start=2022, value=2022, width=225)
    [3] FloatInput(end=25, name='Input Duration (hours)', sizing_mode='stretch_width', start=0, value=1, width=225)
    [4] Select(options=['2', '5', '10', ...], sizing_mode='stretch_width', value='2', width=225)
    [5] FloatInput(end=57.984, name='Select Latitude', sizing_mode='stretch_width', start=41.6, value=54.3829, width=225)
    [6] FloatInput(end=-74.0, name='Select Longitude', sizing_mode='stretch_width', start=-95.96, value=-85.467, width=225)

In [18]:
##Generating choropleth map based on user specified return period, duration, and year
@pn.depends(returnperiod.param.value,duration.param.value,endyear.param.value,var.param.value) #links function to widgets
def create_map_test(
    rp = returnperiod.param.value,
    d = duration.param.value,
    y = endyear.param.value,
    v = var.param.value
):
   #Generating plot, title and description specific to selected variable (v)
    if v == 'MTO 2010 Rainfall Intensity':
        a = "a" + rp #Generating column name related to specified return period to access 'a' parameter data in xarray
        RR2010 = MTOparam_ds[a]*np.power(d,-0.699) #converting a and b parameters to rainfall intensity. b is a constant of -0.699
        title = F"MTO 2010 Rainfall Intensity (mm/hour) [{d} Hour Duration {rp} Year Return Period Storm Event]"
        plot = RR2010.hvplot(kind='image',min_width = 600, height = 350, fontscale = 1,title = title) 
        description = pn.pane.Markdown("""
        This map displays the variation in 2010 rainfall intensity values across Ontario for the specified storm event.
        Rainfall intensity values are represented by the range of colours summarized in the colour bar to the right of the 
        plot. As depicted on this scale, a darker blue corresponds to a higher rainfall intensity, while a lighter blue 
        corresponds to a lower rainfall intensity. The colour white represents areas where no data is provided through this
        tool. Hover the cursor over the map to see the exact 2010 rainfall intensity at any point. 
        </span> """)

    elif v == 'MTO Intensity Multiplier':
        a = "a" + rp #Generating column name related to specified return period
        RR2010 = MTOparam_ds[a]*np.power(d,-0.699) #converting a and b parameters to rainfall intensity. b is a constant of -0.699
        adj = (0.02789*np.power(d,-0.49378))*(y-2010) #finding MTO linear adjustment factor m*(year - 2010)
        RRy = RR2010 + adj #finding future rainfall intensity
        MTOaf = RRy/RR2010 #finding MTO multiplier
        title = F"Low Intensity Multiplier From 2010 to {y} [{d} Hour Duration {rp} Year Return Period Storm Event]"
        plot = MTOaf.hvplot(kind='image',min_width = 600, height = 350, fontscale = 1, title = title)
        description = pn.pane.Markdown("""
        This map displays the variation in MTO intensity multipliers across Ontario, which can be multiplied with 2010 
        rainfall intensity data to project the MTO rainfall intensity at the specified end year. The MTO multiplier 
        represents the linear climate change adjustment MTO currently uses to predict future rainfall intensities. The 
        multipliers are represented on the map by the range of colours summarized in the colour bar to the right of the plot.
        As depicted on this scale, a darker blue corresponds to a higher multiplier, while a lighter blue corresponds to a
        lower multiplier. White areas represent where no data is available through this tool. Hover the cursor over the map
        to see the exact low multiplier at any point.
        </span> """)
    
    #Generating title and description specific to selected temperature variable (v)
    else:
        if v == 'Moderate Projected Temperature Change':
            title = F"{v} (°C) from 2010 to {y} [RCP 4.5]"
            description = pn.pane.Markdown("""
            This map displays the projected changes in temperature across Ontario under a moderate emissions scenario 
            (RCP 4.5) from 2010 to the specified end year. Temperature change values are represented by the range of colours
            summarized in the colour bar to the right of the plot. As depicted on this scale, a darker blue corresponds to 
            a greater increase in temperature, while a lighter blue corresponds to a lower increase in temperature. The 
            colour white represents areas where no data is provided through this tool. Hover the cursor over the map to see
            the exact moderate projected change in temperature at any point. 
            </span> """)
        elif v == 'High Projected Temperature Change':
            title = F"{v} (°C) from 2010 to {y} [RCP 8.5]"
            description = pn.pane.Markdown("""
            This map displays the projected changes in temperature across Ontario under a high emissions scenario (RCP 8.5)
            from 2010 to the specified end year. Temperature change values are represented by the range of colours summarized
            in the colour bar to the right of the plot. As depicted on this scale, a darker blue corresponds to a greater 
            increase in temperature, while a lighter blue corresponds to a lower increase in temperature. The colour white
            represents areas where no data is provided through this tool. Hover the cursor over the map to see the exact
            high projected change in temperature at any point. 
            </span> """)
        elif v == 'Low Intensity Multiplier':
            title = F"{v} from 2010 to {y}"
            description = pn.pane.Markdown("""
            This map displays the variation in low intensity multipliers across Ontario, which can be multiplied with 
            2010 rainfall intensity data to project the low rainfall intensity at the specified end year. The 
            term low represents a climate change adjustment related to the projected change in temperature under a 
            moderate emissions scenario (RCP 4.5). The multipliers are represented on the map by the range of colours 
            summarized in the colour bar to the right of the plot. As depicted on this scale, a darker blue corresponds to a 
            higher multiplier, while a lighter blue corresponds to a lower multiplier. White areas represent where no data
            is available through this tool. Hover the cursor over the map to see the exact medium multiplier at any point.
            </span> """)
        elif v == 'High Intensity Multiplier':
            title = F"{v} from 2010 to {y}"
            description = pn.pane.Markdown("""
            This map displays the variation in high intensity multipliers across Ontario, which can be multiplied with 2010
            rainfall intensity data to project the high rainfall intensity at the specified end year. The term
            high represents a climate change adjustment related to the projected change in temperature under a high emissions
            scenario (RCP 8.5). The multipliers are represented on the map by the range of colours summarized in the colour
            bar to the right of the plot. As depicted on this scale, a darker blue corresponds to a higher multiplier, while
            a lighter blue corresponds to a lower multiplier. White areas represent where no data is available through this
            tool. Hover the cursor over the map to see the exact high multiplier at any point.
            </span> """)
            
        #Generating plot for temperature variable
        #Converting user friendly variable names displayed in tool to names used in code to index datasets (term z = v1 in hvplot)
        l1 = var_userfriendly.index(v) 
        v1 = var_names[l1]
        plot = IDFdata.interactive.sel(time = y).hvplot(kind='image',z=v1,min_width = 600, height = 350, fontscale = 1, title = title).panel()

    display = pn.Column(pn.Column(plot, margin = (0,140,4,140)),description) #formatting choropleth display
    return (
       pn.Card(display,title = "Choropleth Map") #formatting choropleth display
    )
#pn.Column(control_panel,create_map_test).show() #uncomment to test this section individually in the notebook

In [19]:
#Generate summary table based on user specified return period, duration, year, and coordinates
@pn.depends(returnperiod.param.value,duration.param.value,endyear.param.value,lat.param.value,lon.param.value) #Links function to widgets
def SummaryTable(
    rp = returnperiod.param.value,
    d = duration.param.value,
    y = endyear.param.value,
    la = lat.param.value,
    lo = lon.param.value
):
    #Extracting and solving for relevant data
    arp = "a" + rp #Generating column name related to specified return period
    A = MTOparam_ds[arp].sel(lat=la,lon=lo,method = "nearest").values #Extracting A value at selected return period and location
    RR2010 = A*(d**-0.699) #Generates 2010 rainfall intensity
    adj = (0.02789*np.power(d,-0.49378))*(y-2010) #Generates linear adjustment factor to specified endyear
    RRaf = (RR2010 + adj)/RR2010 #Finds MTO multiplier
    dT45 = IDFdata.dT_rcp45.sel(time = y,lat=la,lon=lo,method = "nearest").values.round(decimals = 3) #Extracting moderate change in temp to selected endyear at selected location
    dT85 = IDFdata.dT_rcp85.sel(time = y,lat=la,lon=lo,method = "nearest").values.round(decimals = 3) #Extracting high change in temp to selected endyear at selected locatoin
    af45 = IDFdata.af_rcp45.sel(time = y,lat=la,lon=lo,method = "nearest").values.round(decimals = 3) #Extracting low multiplier to selected endyear at selected location
    af85 = IDFdata.af_rcp85.sel(time = y,lat=la,lon=lo,method = "nearest").values.round(decimals = 3) #Extracting high multiplier to selected endyear at selected location
    
    #Generating interactive column names
    MTO = F"MTO {y} Rainfall Intensity" 
    Low = F"Low {y} Rainfall Intensity"
    High = F"High {y} Rainfall Intensity"
    
    #Creating and formatting a Pandas dataframe with data and appropriate column names
    rows = [{"MTO 2010 Rainfall Intensity":RR2010,"Moderate Temp Increase (C)":dT45,"High Temp Increase (C)":dT85,"MTO Multiplier":RRaf,MTO:(RRaf*RR2010),"Low Multiplier":af45,Low:(af45*RR2010),"High Multiplier":af85,High:(af85*RR2010)}]
    title = f"### Summary of Projected Rainfall Intensity: 2010 to {y}, point nearest to [{la}, {lo}], {d} hour duration {rp} year return period storm event"
    df = pd.DataFrame(rows).style.hide(axis='index').format(precision = 3).set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    df.set_properties(**{'text-align': 'center'})
    
    #Table description text
    description = pn.pane.Markdown(""" 
    This table summarizes projected future rainfall intensities at the selected location for the specified end year and 
    design storm. Projected rainfall intensities are the product of the 2010 rainfall intensity and the respective multiplier.
    """)
    
    return (
        pn.Card(title,pn.pane.DataFrame(df,width = 800),description,title = "Summary Table") #Formatting Summary table display
    )

In [20]:
## Formatting content as webpage
material = pn.template.MaterialTemplate(title='Ontario IDF Climate Change Adjustment Comparison Tool',header_background = '#000080',sidebar_width = 325)

material.sidebar.append(control_panel)

#material.main.append((pn.Tabs(('Tool',pn.Column(overview,create_map_test,SummaryTable)),('About',about)))) }uncomment to show about section, and comment line below.
material.main.append(pn.Column(overview,create_map_test,SummaryTable))

material.show(threaded=True)
#material.servable() #uncomment to show in notebook

<StoppableThread(Thread-17 (get_server), started 6808)>

Launching server at http://localhost:56999
